In [1]:
import numpy as np
import pandas as pd
import os 
import io

In [ ]:
# read the email information from local directory

In [2]:
def read (path):
    for root, dirnames, filenames in os.walk(path):
        for i in filenames:
            path=os.path.join(root,i)
            
            inBody=False
            line=[]
            
            word=io.open(path,"r",encoding="latin1")
            for i in word:
                if inBody:
                    line.append(i)
                elif i == '\n':
                    inBody=True
            word.close()
            message='\n'.join(line)
            yield path,message
                    
def create_dataset(path,classification):
    rows=[]
    index=[]
    for path_01, message in read(path):
        rows.append({"Message":message,"Classification":classification})
        index.append(path_01)
    return pd.DataFrame(rows,index=index)

In [3]:
data=pd.DataFrame({"Message":[],"Classification":[]})
data=data.append(create_dataset(("C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/spam"),"spam"))
data=data.append(create_dataset(("C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/ham"),"real"))

In [4]:
data

,Message,Classification
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam
...,...,...
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/ham\02496.aae0c81581895acfe65323f344340856,Man killed 'trying to surf' on Tube train \n\n...,real
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/ham\02497.60497db0a06c2132ec2374b2898084d3,"Hi Gianni,\n\n\n\nA very good resource for thi...",real
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/ham\02498.09835f512f156da210efb99fcc523e21,Gianni Ponzi wrote:\n\n> I have a prob when tr...,real
C:/Users/caiyangpipo/Desktop/Big Data/ML&DS&DL Python/emails/ham\02499.b4af165650f138b10f9941f6cc5bce3c,Neale Pickett <neale@woozle.org> writes:\n\n\n...,real


In [5]:
data.describe()

,Message,Classification
count,3000,3000
unique,2943,2
top,------000000000000000000000\n\nContent-Type: t...,real
freq,4,2500


In [6]:
data["Classification"].unique()

array(['spam', 'real'], dtype=object)

In [7]:
data.groupby("Classification").count()

,Message
Classification,
real,2500
spam,500


In [ ]:
# create the prediction model on training data

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [9]:
x_train,x_test,y_train,y_test=train_test_split(data["Message"],data["Classification"],test_size=0.2,random_state=365)

In [10]:
vect=CountVectorizer()
text_train=vect.fit_transform(x_train.values)

In [11]:
classifier=MultinomialNB()
classifier.fit(text_train,y_train.values)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# check the accuracy rate on test data

In [12]:
text_test=vect.transform(x_test.values)
prediction=classifier.predict(text_test)

In [13]:
Confusion_Matrix=confusion_matrix(y_test.values,prediction)

In [14]:
Confusion_Matrix

array([[481,   1],
       [ 24,  94]], dtype=int64)

In [22]:
cm=pd.DataFrame(Confusion_Matrix)
cm

,0,1
0,481,1
1,24,94


In [35]:
Accuracy=round((cm[0][0]+cm[1][1])/600*100,2)

In [36]:
print("{}%".format(Accuracy))

95.83%
